In [ ]:

from patsy import dmatrix

import numpy as np
import pandas as pd
import scipy
from scipy.integrate import cumulative_trapezoid as cumtrapz
import matplotlib.pyplot as plt
from statsmodels.gam.api import BSplines
from sklearn.metrics import r2_score

from PacTimeOrig.Attractor import base as attractor
from PacTimeOrig.utils.processing import zscore
from keras.models import Sequential
from keras.layers import Dense
from sklearn.preprocessing import StandardScaler
import numpy as np
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from tensorflow.keras.regularizers import l2,l1




data_dict=attractor.data_loader()
sess='all'
rwdlev=0
data,neural = attractor.data_gather(data_dict,sess=sess)
indata=data['sess_'+str(1)].copy()
indata=indata[indata['reward']==rwdlev].drop(columns=['reward'],axis=1)
Xtrain,Ytrain=attractor.data_organizer_by_model(indata, cfg = {'model_type':'RNN', 'org_by':'trial','data_type':'behavior','trial_drop':True},var_names=['x','time','reward','reldist','relspeed'])

indata=neural['sess_'+str(1)].copy()
indata=indata[indata['reward']==rwdlev].drop(columns=['reward'],axis=1)
Xtrainneural,Ytrain=attractor.data_organizer_by_model(indata, cfg = {'model_type':'RNN', 'org_by':'trial','data_type':'neural','trial_drop':True},var_names=['x','timenormalized','reldist','relspeed'])

XXt=[]
for i in range(len(Xtrainneural)):
    XXt.append(np.hstack([Xtrain[i],Xtrainneural[i]]))

    Code the formula design matrix in patsy, drop intercept and use onehotencoding

In [ ]:
sess=2

indata=data['sess_'+str(sess)].copy()

indata['time'] = pd.cut(indata['time'], bins=21,labels=False)-10

#onehotencode
indata = pd.get_dummies(indata, columns = ['reward'])
indata.rename(columns={'reward_-1.0':'rwd1','reward_-2.0':'rwd2', 'reward_0.0':'rwd3','reward_1.0':'rwd4','reward_2.0':'rwd5'},inplace=True)

inneural=neural['sess_'+str(sess)].copy()
inneural.drop(['x','y','trialnumber','reward'],axis=1,inplace=True)
#Make the patsy formula
formula='x+time+reldist+rwd1+rwd2+rwd3+rwd4+rwd5-1'

X_train=dmatrix(formula, indata)
X_train[:,0]=(X_train[:,0]/5)
X_train[:,1]=(X_train[:,1]/10)
# X_train=np.hstack([X_train,zscore(np.array(inneural))])



Ytrain=(indata['y'])

input_size=X_train.shape[1]
# Compile the model with a custom learning rate
learning_rate = 0.0001  # Change this value to your desired learning rate
optimizer = Adam(learning_rate=learning_rate)
model = Sequential()
model.add(Dense(128, input_dim=input_size, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(1,activation='linear'))

model.compile(optimizer=optimizer,  loss='mean_squared_error')


history=model.fit(X_train,zscore(Ytrain) ,epochs=25, batch_size=32,validation_split=0.1)


plt.plot(history.history['loss'], label='Training Accuracy')
plt.plot(history.history['val_loss'], label='Validation Accuracy')



# Make predictor time

In [ ]:
sz=101
nlevtimes=np.linspace(-1,1,91)


output=np.zeros((100,91,5,5))

for reliter,reldist in enumerate([-1,-0.5,0,0.5,1]):
    for rwdcol in range(5):
        for timeiter, times in enumerate(nlevtimes):
            Xpredictor=np.zeros((input_size,sz))
            Xpredictor[0,:]=np.linspace(np.min(X_train[:,0]),np.max(X_train[:,0]),sz)
            Xpredictor[1,:]=(times*np.ones((sz,1))).flatten()
            #Mean rel dist
            Xpredictor[2,:]=((reldist)*np.ones((sz,1))).flatten()
            #Mean rel speed
            rwd=np.zeros((5,sz))
            rwd[rwdcol,:]=1
            Xpredictor[3:,:]=rwd
            output[:,timeiter,rwdcol,reliter]=-cumtrapz(model.predict(Xpredictor.transpose()).flatten()).flatten()
        


With an RNN

In [ ]:
# Create the data generator
# Build the model
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(None, 5)),  # Input shape with None for variable length
    tf.keras.layers.LSTM(4, activation='tanh', return_sequences=True),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(1, activation='linear'))
])


batch_size = 5
val_batch_size = 20
val_data_gen = attractor.VariableLengthDataGenerator(Xtrain[201:], Ytrain[201:], val_batch_size)
data_gen=attractor.VariableLengthDataGenerator(Xtrain[0:200],Ytrain[0:200],batch_size)

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model using the data generator
model.fit(data_gen,validation_data=val_data_gen, epochs=30)

trial=2
single_trial = np.expand_dims(Xtrain[trial], axis=0)
plt.plot(zscore(model.predict(single_trial)[0]))
plt.plot(zscore(Ytrain[trial]))
plt.show()